# Data augmentation with late merge and hard parameters #

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import random
from sklearn.metrics import accuracy_score

we will be making a function to make a new directory with random augmented images that we will then feed to the dataloader to get both the original dataset and the new augmented image dataset for both classes in this case

In [2]:
def make_augmented_directory(old_path, newpath):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
        transforms.ToTensor()
    ])
    classes = [i for i in os.listdir(old_path) if i != '.DS_Store']
    for c in classes:
        this_class_path = os.path.join(old_path, c)
        all_folders_in_this_class = [i for i in os.listdir(this_class_path) if i!= '.DS_Store']
        for folder in all_folders_in_this_class:
            this_folder_path = os.path.join(this_class_path, folder)
            images_in_this_folder = os.listdir(this_folder_path)
            for image_f in images_in_this_folder:
                image_path = os.path.join(this_folder_path, image_f)
                img = Image.open(image_path)
                augmented_image = transform(img)
                augmented_image = transforms.ToPILImage()(augmented_image)
                save_dir = os.path.join(newpath, c, folder)
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                save_path = os.path.join(save_dir, f'{image_f}')
                augmented_image.save(save_path)
                


In [3]:
make_augmented_directory("./dataset/", "./augmented_dataset/")

In [4]:
print(len(os.listdir('./augmented_dataset')))
print(len(os.listdir('./augmented_dataset/Snow')))
print(len(os.listdir('./augmented_dataset/NotSnow')))

2
1049
2517


In [5]:
class data_aug_dataloader(Dataset):
    def __init__(self, main_data_dir ,secondary_data_dir, train):
        self.data_dir = [main_data_dir, secondary_data_dir]
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.dataset_maker(train)
        self.length = len(self.dataset)
    def dataset_maker(self, train):
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.data_dir[0], class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.data_dir[1], class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        ])
        pic1 = transform(pic1)
        pic2 = transform(pic2)
        return pic1 , pic2, sample_class

In [6]:
data_aug_training = data_aug_dataloader("./dataset/", "./augmented_dataset/", True)
data_aug_testing = data_aug_dataloader("./dataset/", "./augmented_dataset/", False)

In [7]:
sample = data_aug_training[1311]
print(sample[0].shape)
print(sample[1].shape)
print(sample[2])

torch.Size([1, 224, 224])
torch.Size([1, 224, 224])
0


In [8]:
print(f'the new testing set has {len(data_aug_testing)} samples')
print(f'the new training set has {len(data_aug_training)} samples')


the new testing set has 1427 samples
the new training set has 5705 samples


In [9]:
class late_merge_NN(nn.Module):
    def __init__(self):
        super(late_merge_NN, self).__init__()
        self.convolution1 = nn.Conv2d(1, 2, 1)
        self.convolution2 = nn.Conv2d(2, 4, 1)
        self.pool = nn.MaxPool2d(kernel_size=4, stride=5)
        self.convolution12 = nn.Conv2d(1, 2, 1)
        self.convolution22 = nn.Conv2d(2, 4, 1)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=5)
        self.full1 = nn.Linear(648, 2)

    def forward(self, x1, x2):
        x1 = self.convolution1(x1)
        x1 = self.pool(x1)
        x1 = self.convolution2(x1)
        x1 = self.pool(x1)
        x1 = x1.view(-1, 324)
        
        x2 = self.convolution12(x2)
        x2 = self.pool2(x2)
        x2 = self.convolution22(x2)
        x2 = self.pool2(x2)
        x2 = x2.view(-1, 324)

        x = torch.cat((x1, x2), dim=1)
        x = self.full1(x)
        return x


In [10]:
# hyper parameters
learning_rate_7= 0.001
epochs_7 = 3

In [11]:
# training the neural net 
model7 = late_merge_NN()
criterion7 = nn.CrossEntropyLoss()
optimiser7 = optim.Adam(model7.parameters(), lr = learning_rate_7)


In [12]:
# model params
total_params = sum(p.numel() for p in model7.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 1330


In [13]:
model7.train()
for epoch in range(epochs_7):
    running_loss = 0 
    predicted_list = []
    labels_list = []
    print(f'this is epoch number {epoch}')
    for i, (input1,input2, label) in enumerate(data_aug_training):
        optimiser7.zero_grad()
        labels_list.append(label)
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model7(input1, input2)
        loss = criterion7(outputs, label)
        loss.backward()
        optimiser7.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        if i % 1000 == 999:   
            print('loss: %.3f accuracy: %.3f' %(running_loss / 10, 100 * accuracy_score(labels_list, predicted_list)))
            running_loss = 0

this is epoch number 0
this is the 0 iteration
loss: 48.515 accuracy: 75.200
this is the 1000 iteration
loss: 34.260 accuracy: 80.300
this is the 2000 iteration
loss: 27.527 accuracy: 83.367
this is the 3000 iteration
loss: 26.820 accuracy: 84.800
this is the 4000 iteration
loss: 26.843 accuracy: 85.680
this is the 5000 iteration
this is epoch number 1
this is the 0 iteration
loss: 20.115 accuracy: 92.700
this is the 1000 iteration
loss: 24.770 accuracy: 91.850
this is the 2000 iteration
loss: 22.845 accuracy: 92.100
this is the 3000 iteration
loss: 23.001 accuracy: 91.700
this is the 4000 iteration
loss: 23.420 accuracy: 91.660
this is the 5000 iteration
this is epoch number 2
this is the 0 iteration
loss: 18.056 accuracy: 93.900
this is the 1000 iteration
loss: 22.835 accuracy: 93.100
this is the 2000 iteration
loss: 21.528 accuracy: 93.167
this is the 3000 iteration
loss: 21.162 accuracy: 92.650
this is the 4000 iteration
loss: 21.708 accuracy: 92.620
this is the 5000 iteration


model7.eval()
loss = 0 
predicted_list = []
labels_list = []
with torch.no_grad():
    for i, (input1,input2, label) in enumerate(data_aug_testing):
        label = torch.tensor(label)
        label = torch.tensor(label).view(-1)
        labels_list.append(label.item())
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model7(input1, input2)
        loss += criterion7(outputs, label).item()
        predicted = outputs.argmax(dim = 1 , keepdim = True)
        predicted_list.append(predicted.item())

In [15]:
print(f'the model accuracy is {accuracy_score(labels_list, predicted_list)}')

the model accuracy is 0.9264190609670637
